In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
# import ydf
import tensorflow_decision_forests as tfdf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs found: {gpus}")
else:
    print("No GPUs found.")

2025-10-11 11:47:55.714829: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760183276.223549      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760183276.392742      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPUs found: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
train_df=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_df=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
# preprocess?
def preprocess(df):
    # def cryoSleep_integer(x):
    #     return x.astype(int)
    # df=df.dropna()
    def normalize_name(x):
        x=str(x)
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    def split_passenger_id(row):
        if pd.isna(row['PassengerId']):
            return pd.Series([None, None, None])
        arr=row['PassengerId'].split('_')
        if len(arr) < 2:
            arr += [None] * (3 - len(arr))
        
        return pd.Series(arr)
    def split_cabin(row):
        if pd.isna(row['Cabin']):
            return pd.Series([None, None, None])  # Handle None case
        arr = row['Cabin'].split('/')
        # Ensure arr has exactly 3 elements
        if len(arr) < 3:
            arr += [None] * (3 - len(arr))  # Pad with None
        return pd.Series(arr)
    def total_spent(row):
        return pd.Series(row['RoomService']+row['FoodCourt']+row['ShoppingMall']+row['Spa']+row['VRDeck'])
        

    # Apply the function and create new columns
    df.fillna({
    'RoomService': df['RoomService'].median(),
    'FoodCourt': df['FoodCourt'].median(),
    'ShoppingMall':df['ShoppingMall'].median(),
    'Spa':df['Spa'].median(),
    'VRDeck':df['VRDeck'].median(),
    # 'Deck':df['Deck'].median()
    }, inplace=True)
    df['Total_Spent']=df.apply(total_spent,axis=1)
    df["Name"] = df["Name"].apply(normalize_name)
    df[['Deck', 'Num', 'Side']] = df.apply(split_cabin, axis=1)
    df[['Group','Place_In_Group']]=df.apply(split_passenger_id,axis=1)
    df['CryoSleep'] = df['CryoSleep'].apply(lambda x: float(x))
    df=df.fillna({'CryoSleep':df['CryoSleep'].mean()})
    df['VIP'] = df['VIP'].apply(lambda x: float(x))

    
    columns_to_convert = df.columns.difference(['PassengerId'])
    
    # Convert selected columns to numeric while handling non-convertible values
    df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')    
    
    return df
    
train_df=preprocess(train_df)
test_df=preprocess(test_df)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     8693 non-null   object 
 1   HomePlanet      0 non-null      float64
 2   CryoSleep       8693 non-null   float64
 3   Cabin           0 non-null      float64
 4   Destination     0 non-null      float64
 5   Age             8514 non-null   float64
 6   VIP             8490 non-null   float64
 7   RoomService     8693 non-null   float64
 8   FoodCourt       8693 non-null   float64
 9   ShoppingMall    8693 non-null   float64
 10  Spa             8693 non-null   float64
 11  VRDeck          8693 non-null   float64
 12  Name            0 non-null      float64
 13  Transported     8693 non-null   bool   
 14  Total_Spent     8693 non-null   float64
 15  Deck            0 non-null      float64
 16  Num             8494 non-null   float64
 17  Side            0 non-null      f

In [5]:
train_df.head(5)

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Total_Spent,Deck,Num,Side,Group,Place_In_Group
0,0001_01,NaN,0.0,NaN,NaN,39.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,False,0.0,NaN,0.0,NaN,1,1
1,0002_01,NaN,0.0,NaN,NaN,24.0,0.0,109.0,9.0,25.0,549.0,44.0,NaN,True,736.0,NaN,0.0,NaN,2,1
2,0003_01,NaN,0.0,NaN,NaN,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,NaN,False,10383.0,NaN,0.0,NaN,3,1
3,0003_02,NaN,0.0,NaN,NaN,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,NaN,False,5176.0,NaN,0.0,NaN,3,2
4,0004_01,NaN,0.0,NaN,NaN,16.0,0.0,303.0,70.0,151.0,565.0,2.0,NaN,True,1091.0,NaN,1.0,NaN,4,1


In [6]:
train_df=train_df.fillna(0)

In [7]:

input_features=list(train_df.columns)
input_features.remove('Transported')
input_features.remove('PassengerId')
input_features.remove('Cabin')
# input_features=['CryoSleep']
# input_features.remove('Name')
# imput_features.remove('')

In [8]:
from sklearn.preprocessing import StandardScaler



# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(len(input_features),)),
    tf.keras.layers.Dense(32, activation='relu'),  # Increased complexity
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='SGD', 
              loss = tf.keras.losses.BinaryFocalCrossentropy(),
              metrics=['accuracy'])

# Prepare the training data
X_train = train_df[input_features].values
y_train = train_df['Transported'].astype(int).values
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Fit the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

# To predict on the test set
X_test = test_df[input_features].values
predictions = model.predict(X_test)

I0000 00:00:1760183302.052120      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760183302.052774      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/50


I0000 00:00:1760183304.762353      62 service.cc:148] XLA service 0x7c95f4005330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760183304.763660      62 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760183304.763677      62 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760183305.014274      62 cuda_dnn.cc:529] Loaded cuDNN version 90300


 79/218 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5106 - loss: 0.2986

I0000 00:00:1760183305.807418      62 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5205 - loss: 0.2554 - val_accuracy: 0.6164 - val_loss: 0.1731
Epoch 2/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6716 - loss: 0.1629 - val_accuracy: 0.7159 - val_loss: 0.1584
Epoch 3/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7131 - loss: 0.1513 - val_accuracy: 0.7418 - val_loss: 0.1507
Epoch 4/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7319 - loss: 0.1447 - val_accuracy: 0.7510 - val_loss: 0.1451
Epoch 5/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7448 - loss: 0.1379 - val_accuracy: 0.7596 - val_loss: 0.1409
Epoch 6/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7505 - loss: 0.1365 - val_accuracy: 0.7608 - val_loss: 0.1381
Epoch 7/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7696 - loss: 0.1311 - val_accuracy: 0.7637 - val_loss: 0.1362
Epoch 8/50
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7740 - loss: 0.1296 - val_accuracy: 0.7654 - val_

In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,123 (4.39 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [10]:
def prediction_to_kaggle_format(model, test_df, threshold=0.5):
    # Prepare features for predictions using the same input features as used in training
    X_test = test_df[input_features].values  # Ensure input features are consistent
    proba_survive = model.predict(X_test, verbose=0)  # Predict probabilities
    
    # Check the shape of proba_survive
    if proba_survive.ndim > 1:
        proba_survive = proba_survive[:, 0]  # Flatten if necessary
    
    # Create the DataFrame for submission
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"].values,  # Ensure this is a 1D array
        "Transported": (proba_survive >= threshold).astype(str)  # Convert boolean to string
    })

def make_submission(kaggle_predictions):
    path = "/kaggle/working/submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

# Generate predictions and make the submission
kaggle_predictions = prediction_to_kaggle_format(model, test_df)  # Use preprocessed test_df
make_submission(kaggle_predictions)

# Optional: Check the first few lines of the submission
!head /kaggle/working/submission.csv

Submission exported to /kaggle/working/submission.csv
PassengerId,Transported
0013_01,False
0018_01,False
0019_01,False
0021_01,False
0023_01,False
0027_01,False
0029_01,False
0032_01,False
0032_02,False


/tmp/ipykernel_19/483588305.py:13: RuntimeWarning: invalid value encountered in greater_equal
  "Transported": (proba_survive >= threshold).astype(str)  # Convert boolean to string
